# Connecting to Google API


In [1]:
#import libraries
import pandas as pd 
import gspread 

In [2]:
#taking Google APi credentials
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./daily-articles.json', scope) 
gc = gspread.authorize(credentials)

In [4]:
#google spreadsheet key in share url
spreadsheet_key = '1I2huckhoF6dcffty9VCGf2vHULdmfvp_4yIZAxumkIw' 

# Requesting Info from Naver News

In [5]:
#importing libraries for web scraping 
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
from datetime import date
#import library for connecting Google Sheets and Jupyter
from df2gspread import df2gspread as d2g

In [6]:
#list of keywords we will use
keywords = ["터키리라","리라화","터키환율","해외투자","해외진출","터키 무역","터키 스타트업","터키 바이오","터키 첨단산업","터키 경제"]

In [7]:

url_list = []
key_list = {}
for n in np.arange(0, len(keywords)):
    #getting page content for each keyword
    url_each = "https://m.search.naver.com/search.naver?where=m_news&query="+keywords[n]+"&sm=mtb_opt&sort=1&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Add%2Cp%3Aall&is_sug_officeid=0"
    #this url uses "최신순" sorting
    url_list.append(url_each)
    r = requests.get(url_each)
    page = r.content
    soup = BeautifulSoup(page, 'html5lib')
    news = soup.find_all('a', class_='news_tit')
    if keywords[n] not in key_list:
        key_list[keywords[n]] = news


In [8]:
big_list = [["Keyword","Date","Article","Link"]]
for key in key_list:
      
    #getting details for each keyword
    number_of_articles = len(key_list[key]) #default 15 articles per page
    list_links = []
    list_titles = []
   
    for n in np.arange(0, number_of_articles):
    
            # Getting the link of the article
        link = news[n]['href']
        list_links.append(link)

            # Getting the title
        title = news[n].get_text()
        list_titles.append(title)
    #     big_list.append(key:{
    #         "Articles" :list_titles,
    #         "Links":list_links
    #     })
    #     big_list[key] = {
    #         "Articles":list_titles,
    #          "Links":list_links}
        big_list.append([key,str(date.today()),title, link])

In [9]:
#big array of 10(keywords)*15(articles per keyword) => 150 rows
len(big_list)

151

# Sending Data to Google Sheets

In [10]:
today = date.today()
wks_name = str(today) + ' Articles'
table =  pd.DataFrame(big_list)
d2g.upload(table, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet '2021-06-17 Articles' id:1792356366>

# Unused code

In [61]:
# def makeTable(key):
#     data_keyword = pd.DataFrame({
#     "Keyword": key,
#     "Date": str(date.today()),
#     'Article Tiiitle': big_list[key]['Articles'],
#     'Article Link': big_list[key]['Links'],
# })
#     return data_keyword

In [26]:
# table = makeTable(keywords[0])
# for n in np.arange(1, len(keywords)):
#     table2 = makeTable(keywords[n])
#     table = table.append(table2)
    
# print(table)


In [110]:
# df = table
# df = df.applymap(str)
# df = df.groupby('Keyword')
